# **Transcription with Conversation-Level Sentiment Annotations**
Below, we use the Hume and GPT APIs to generate conversation-level sentiment annotations for a Zoom conversation.

> We design a two-part pipeline to visualize Zoom meetings with conversation-level sentiment annotations. We first introduce novel metrics to capture conversation-level sentiments along three axes: comprehension, consensus, and cordiality. To obtain these metrics, we first identify each speaker's individual expressed sentiments during each of their responses. To determine speaker sentiment, we segment Zoom recordings by speaker and feed the video data, audio file (including information on voice prosity), and transcript (text content) of each segment to an off-the-shelf model that outputs a quantitative measure of the extent to which the speaker expresses 48 emotions. Afterward, for each segment, we combine the speaker's top 5 emotions with weights, uniformly sampled facial expressions, and spoken words in an instruction-tuned prompt to a multimodal large language model in order to determine conversation-level metrics.



# Initialization

In [42]:
# Install libraries
!pip install hume
!pip install hume[stream]
!pip install openai
!pip install python-dotenv
!pip install pydub
!pip install ffmpeg
!pip install moviepy
!pip install webvtt-py
!pip install opencv-python


In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import requests
import base64
from pydub import AudioSegment
from hume import HumeBatchClient
import json
import pandas as pd
import matplotlib.pyplot as plt
# from google.colab import userdata
import webvtt
# from google.colab import userdata
import cv2
from moviepy.editor import VideoFileClip
import subprocess

## Merge three Modalities

## 1. Merge language to sentenses, and average scores for each emotions

In [19]:
import os
import json

def process_json_file(filepath, output_directory):
    # Determine output file path
    output_path = os.path.join(output_directory, os.path.basename(filepath).replace('_lang.json', '_lang_processed.json'))
    
    # Skip processing if the file already exists
    if os.path.exists(output_path):
        print(f"Skipping existing file: {output_path}")
        return
    
    # Read JSON file
    with open(filepath, 'r') as file:
        data = json.load(file)
    
    # Navigate to the predictions in JSON structure
    predictions = data[0]["results"]["predictions"][0]["models"]["language"]["grouped_predictions"][0]["predictions"]

    # Concatenate text to form full sentence
    full_sentence = " ".join([pred["text"] for pred in predictions])
    
    # Calculate average emotion scores
    emotion_scores = {}
    count_emotions = {}
    for pred in predictions:
        for emotion in pred["emotions"]:
            if emotion["name"] in emotion_scores:
                emotion_scores[emotion["name"]] += emotion["score"]
                count_emotions[emotion["name"]] += 1
            else:
                emotion_scores[emotion["name"]] = emotion["score"]
                count_emotions[emotion["name"]] = 1
    
    # Averaging the scores
    average_emotion_scores = {emotion: score / count_emotions[emotion] for emotion, score in emotion_scores.items()}
    
    # Save processed data to a new file
    with open(output_path, 'w') as outfile:
        json.dump({'sentence': full_sentence, 'emotions': average_emotion_scores}, outfile, indent=4)
    print(f"Processed and saved: {output_path}")

# Directories
input_directory = "./dataset/outputs/hume"
output_directory = "./dataset/outputs/hume_processed"

# Create the output directory if it does not exist
os.makedirs(output_directory, exist_ok=True)

# List all files with '_lang.json' suffix
json_files = [os.path.join(input_directory, f) for f in os.listdir(input_directory) if f.endswith('_lang.json')]

# Process each JSON file
for file_path in json_files:
    process_json_file(file_path, output_directory)

print("All files processed or skipped if already existing.")


Processed and saved: ./dataset/outputs/hume_processed\113_00-00-00.000_00-00-06.380_Monica_neutral_neutral_lang_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-06.590_00-00-07.092_Rachel_sadness_negative_lang_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-07.092_00-00-09.224_Monica_neutral_neutral_lang_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-09.224_00-00-09.546_Monica_sadness_negative_lang_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-10.385_00-00-16.933_Monica_neutral_neutral_lang_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-18.602_00-00-21.145_Monica_neutral_neutral_lang_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-21.939_00-00-26.776_Rachel_surprise_negative_lang_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-27.236_00-00-28.903_Monica_neutral_neutral_

## 2. Merge face expressions to sentenses, and average scores for each emotions

In [18]:
import os
import json

def process_face_json_file(filepath, output_directory):
    # Determine output file path
    output_path = os.path.join(output_directory, os.path.basename(filepath).replace('_face.json', '_face_processed.json'))
    
    # Skip processing if the file already exists
    if os.path.exists(output_path):
        print(f"Skipping existing file: {output_path}")
        return
    
    # Read JSON file
    with open(filepath, 'r') as file:
        data = json.load(file)
    
    # Navigate to the face predictions in JSON structure
    predictions = data[0]["results"]["predictions"]

    # Calculate average emotion scores
    emotion_scores = {}
    count_emotions = {}
    for pred in predictions:
        for model in pred["models"]["face"]["grouped_predictions"]:
            for emotion in model["predictions"][0]["emotions"]:
                if emotion["name"] in emotion_scores:
                    emotion_scores[emotion["name"]] += emotion["score"]
                    count_emotions[emotion["name"]] += 1
                else:
                    emotion_scores[emotion["name"]] = emotion["score"]
                    count_emotions[emotion["name"]] = 1
    
    # Averaging the scores
    average_emotion_scores = {emotion: score / count_emotions[emotion] for emotion, score in emotion_scores.items()}
    
    # Save processed data to a new file
    processed_data = {
        "source": data[0]["source"],
        "results": {
            "predictions": [{
                "file": data[0]["results"]["predictions"][0]["file"],
                "file_type": "video_no_audio",
                "models": {
                    "face": {
                        "metadata": None,
                        "grouped_predictions": [{
                            "id": "average",
                            "predictions": [{
                                "frame": "average",
                                "time": "average",
                                "prob": "average",
                                "box": "average",
                                "emotions": [{"name": k, "score": v} for k, v in average_emotion_scores.items()]
                            }]
                        }]
                    }
                }
            }]
        }
    }

    with open(output_path, 'w') as outfile:
        json.dump(processed_data, outfile, indent=4)
    print(f"Processed and saved: {output_path}")

# Directories
input_directory = "./dataset/outputs/hume"
output_directory = "./dataset/outputs/hume_processed"

# Create the output directory if it does not exist
os.makedirs(output_directory, exist_ok=True)

# List all files with '_face.json' suffix
json_files = [os.path.join(input_directory, f) for f in os.listdir(input_directory) if f.endswith('_face.json')]

# Process each JSON file
for file_path in json_files:
    process_face_json_file(file_path, output_directory)

print("All face files processed or skipped if already existing.")


Processed and saved: ./dataset/outputs/hume_processed\113_00-00-00.000_00-00-06.380_Monica_neutral_neutral_face_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-06.590_00-00-07.092_Rachel_sadness_negative_face_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-07.092_00-00-09.224_Monica_neutral_neutral_face_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-09.224_00-00-09.546_Monica_sadness_negative_face_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-10.385_00-00-16.933_Monica_neutral_neutral_face_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-18.602_00-00-21.145_Monica_neutral_neutral_face_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-21.939_00-00-26.776_Rachel_surprise_negative_face_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-27.236_00-00-28.903_Monica_neutral_neutral_

## process prosody

In [1]:
import os
import json

def process_prosody_json_file(filepath, output_directory):
    # Preset emotions based on typical data structure
    full_emotions_list = [
        "Admiration", "Adoration", "Aesthetic Appreciation", "Amusement", "Anger", "Annoyance",
        "Anxiety", "Awe", "Awkwardness", "Boredom", "Calmness", "Concentration", "Confusion",
        "Contemplation", "Contempt", "Contentment", "Craving", "Desire", "Determination",
        "Disappointment", "Disapproval", "Disgust", "Distress", "Doubt", "Ecstasy", "Embarrassment",
        "Empathic Pain", "Enthusiasm", "Entrancement", "Envy", "Excitement", "Fear", "Gratitude",
        "Guilt", "Horror", "Interest", "Joy", "Love", "Nostalgia", "Pain", "Pride", "Realization",
        "Relief", "Romance", "Sadness", "Sarcasm", "Satisfaction", "Shame", "Surprise (negative)",
        "Surprise (positive)", "Sympathy", "Tiredness", "Triumph"
    ]

    # Determine output file path
    output_path = os.path.join(output_directory, os.path.basename(filepath).replace('_prosody.json', '_prosody_processed.json'))
    
    # Skip processing if the file already exists
    if os.path.exists(output_path):
        print(f"Skipping existing file: {output_path}")
        return
    
    # Read JSON file
    with open(filepath, 'r') as file:
        data = json.load(file)
    
    # Initialize a placeholder for predictions
    predictions = []

    processed_data = {
        "source": data[0]["source"],
        "results": {
            "predictions": [],
            "errors": data[0]["results"].get("errors", [])
        }
    }

    # Process predictions if they exist
    if data[0]["results"]["predictions"]:
        predictions = data[0]["results"]["predictions"][0]["models"]["prosody"]["grouped_predictions"][0]["predictions"]
        
        # Calculate average emotion scores
        emotion_scores = {}
        count_emotions = {}
        for pred in predictions:
            for emotion in pred["emotions"]:
                if emotion["name"] in emotion_scores:
                    emotion_scores[emotion["name"]] += emotion["score"]
                    count_emotions[emotion["name"]] += 1
                else:
                    emotion_scores[emotion["name"]] = emotion["score"]
                    count_emotions[emotion["name"]] = 1

        # Averaging the scores
        average_emotion_scores = {emotion: score / count_emotions[emotion] for emotion, score in emotion_scores.items()}
    else:
        # Handle case with errors by setting all emotion scores to zero
        average_emotion_scores = {emotion: 0 for emotion in full_emotions_list}

    # Text content logic based on the existence of predictions
    text_content = " ".join([p["text"] for p in predictions]) if predictions else "Error: No transcribable content."

    # Create processed predictions with averaged data or zeros in case of errors
    processed_data["results"]["predictions"].append({
        "file": data[0]["source"]["filename"],
        "file_type": "audio",
        "models": {
            "prosody": {
                "metadata": None,
                "grouped_predictions": [{
                    "id": "average",
                    "predictions": [{
                        "text": text_content,
                        "time": {"begin": 0, "end": 0},
                        "confidence": 0,
                        "emotions": [{"name": k, "score": v} for k, v in average_emotion_scores.items()]
                    }]
                }]
            }
        }
    })

    # Save processed data to a new file
    with open(output_path, 'w') as outfile:
        json.dump(processed_data, outfile, indent=4)
    print(f"Processed and saved: {output_path}")

# Directories
input_directory = "./dataset/outputs/hume"
output_directory = "./dataset/outputs/hume_processed"

# Create the output directory if it does not exist
os.makedirs(output_directory, exist_ok=True)

# List all files with '_prosody.json' suffix
json_files = [os.path.join(input_directory, f) for f in os.listdir(input_directory) if f.endswith('_prosody.json')]

# Process each JSON file
for file_path in json_files:
    process_prosody_json_file(file_path, output_directory)

print("All prosody files processed or skipped if already existing.")


Processed and saved: ./dataset/outputs/hume_processed\113_00-00-00.000_00-00-06.380_Monica_neutral_neutral_prosody_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-06.590_00-00-07.092_Rachel_sadness_negative_prosody_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-07.092_00-00-09.224_Monica_neutral_neutral_prosody_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-09.224_00-00-09.546_Monica_sadness_negative_prosody_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-10.385_00-00-16.933_Monica_neutral_neutral_prosody_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-18.602_00-00-21.145_Monica_neutral_neutral_prosody_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-21.939_00-00-26.776_Rachel_surprise_negative_prosody_processed.json
Processed and saved: ./dataset/outputs/hume_processed\113_00-00-27.236_00-00-28.903_Mo

# histogram of word count and length